In [ ]:

import sys
from pathlib import Path
# Add src directory to path to import organoid_analysis package
sys.path.insert(0, str(Path('..') / 'src'))
import napari
import numpy as np
from tqdm import tqdm
import pandas as pd
import pyclesperanto_prototype as cle 
from skimage.measure import regionprops_table
from organoid_analysis.utils import list_images, read_image, extract_scaling_metadata, simulate_cell_chunked_3d

In [49]:
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane")]

In [50]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\F2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\G2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\P

In [51]:
# Initialize Napari Viewer
viewer = napari.Viewer(ndisplay=2)

In [52]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id from filename
well_id = filename.split("_")[0]


Image analyzed: B2


In [53]:
#TODO: Extract number of multipos and loop through that range (store position for data extraction)
# Open one of the multipositions in the img file
img = img[3]
img = img.transpose(1, 0, 2, 3)

In [54]:
# Try Voronoi Otsu for simple nuclei segmentation

# Extract x,y,z scaling from .nd2 file metadata in order to make data isotropic
scaling_x_um, scaling_y_um, scaling_z_um = extract_scaling_metadata(image)

# Adjust so voxel size_x and size_y so they are equal to 1 to avoid compression upon rescaling
multiplier = 1 / scaling_x_um

scaling_x_um = scaling_x_um * multiplier
scaling_y_um = scaling_y_um * multiplier
scaling_z_um = scaling_z_um * multiplier

# Make data isotropic in all channels and rebuild the image data stack, then delete original img variable to free up memory
resampled_channels = []

for channel in range (0, img.shape[0]):
    # Make data isotropic
    resampled = cle.scale(img[channel], factor_x=scaling_x_um, factor_y=scaling_y_um, factor_z=scaling_z_um, auto_size=True)
    resampled_channels.append(resampled)

# Stack into a single array: (C, Z, Y, X)
resampled_stack = np.stack(resampled_channels, axis=0)

# Free up memory
del img

# Add resample stack to Napari to check label accuracy
viewer.add_image(resampled_stack,name="isotropic_stack")

# Apply voronoi_otsu_labeling and segment nuclei
nuclei_labels = cle.voronoi_otsu_labeling(resampled_stack[2])
viewer.add_labels(nuclei_labels)

Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm


<Labels layer 'nuclei_labels' at 0x1b13aa928f0>

In [55]:
# Pull .cle array from GPU memory into Numpy array for later use with skimage-measure
nuclei_labels = cle.pull(nuclei_labels)

# Simulate cell around the nuclei, cytoplasm does not work since cells are very tightly packed
cell_labels = simulate_cell_chunked_3d(nuclei_labels, dilation_radius=5, erosion_radius=0)
viewer.add_labels(cell_labels)

<Labels layer 'cell_labels' at 0x1b3fb9bece0>

In [56]:
markers

# Create a dictionary containing all image descriptors
#TODO: Add multiposition index during BP
descriptor_dict = {
            "filename": filename,
            "well_id": well_id,
            }

In [57]:
props_list = []

# Loop through markers and extract 
for marker_name, ch_nr, location in markers:
    print(f"Analyzing channel: {marker_name}")

    props = regionprops_table(label_image=cell_labels,
                            intensity_image=resampled_stack[ch_nr],
                            properties=[
                                "label",
                                "area",
                                "intensity_mean",
                                "intensity_min",
                                "intensity_max",
                                "intensity_std",
                            ],
                        )
    
    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename intensity_mean column to indicate the specific image
    prefix = f"{location}_{marker_name}"

    rename_map = {
        "intensity_mean": f"{prefix}_mean_int", # concentration proxy
        "intensity_min":  f"{prefix}_min_int",
        "intensity_max":  f"{prefix}_max_int",
        "intensity_std":  f"{prefix}_std_int",
    }

    props_df.rename(columns=rename_map, inplace=True)

    # Max / mean ratio (puncta vs diffuse signal)
    props_df[f"{prefix}_max_mean_ratio"] = (props_df[f"{prefix}_max_int"] /props_df[f"{prefix}_mean_int"])
    # Total marker content per cell
    props_df[f"{prefix}_sum_int"] = (props_df[f"{prefix}_mean_int"] * props_df["area"])

    # Append each props_df to props_list
    props_list.append(props_df)

    # Initialize the df with the first df in the list
    props_df = props_list[0]
    # Start looping from the second df in the list
    for df in props_list[1:]:
        props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0    
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

Analyzing channel: Occludin_RFP
Analyzing channel: Claudin_FITC


In [58]:
props_df

,filename,well_id,label,area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,membrane_Occludin_RFP_sum_int,area_y,membrane_Claudin_FITC_mean_int,membrane_Claudin_FITC_min_int,membrane_Claudin_FITC_max_int,membrane_Claudin_FITC_std_int,membrane_Claudin_FITC_max_mean_ratio,membrane_Claudin_FITC_sum_int
0,B2,B2,1,3915.0,156.379562,97.0,301.0,23.689930,1.924804,6.122260e+05,3915.0,309.938690,125.0,1312.0,116.974625,4.233095,1.213410e+06
1,B2,B2,2,4583.0,155.959198,88.0,265.0,22.578220,1.699162,7.147610e+05,4583.0,302.114563,125.0,999.0,100.959328,3.306693,1.384591e+06
2,B2,B2,3,10582.0,139.609055,74.0,242.0,18.239056,1.733412,1.477343e+06,10582.0,231.031372,84.0,917.0,113.978767,3.969158,2.444774e+06
3,B2,B2,4,7800.0,172.232559,83.0,1270.0,83.398308,7.373751,1.343414e+06,7800.0,249.425507,86.0,917.0,113.956078,3.676448,1.945519e+06
4,B2,B2,5,10417.0,150.447632,88.0,242.0,22.793528,1.608533,1.567213e+06,10417.0,317.182678,88.0,1295.0,155.386139,4.082821,3.304092e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,B2,B2,1133,7313.0,302.889923,161.0,910.0,71.399994,3.004392,2.215034e+06,7313.0,577.415405,243.0,1121.0,152.991089,1.941410,4.222639e+06
1133,B2,B2,1134,13375.0,273.376221,145.0,703.0,52.178200,2.571548,3.656407e+06,13375.0,633.814575,283.0,1518.0,171.467163,2.395022,8.477270e+06
1134,B2,B2,1135,5598.0,286.910492,146.0,704.0,71.063797,2.453727,1.606125e+06,5598.0,744.734375,261.0,2115.0,235.104111,2.839939,4.169023e+06
1135,B2,B2,1136,7430.0,256.597321,125.0,656.0,50.560452,2.556535,1.906518e+06,7430.0,569.261108,206.0,1785.0,189.274658,3.135644,4.229610e+06


In [59]:
import numpy as np
from scipy.ndimage import distance_transform_edt
from skimage.filters import threshold_otsu
from skimage.segmentation import watershed
from skimage.morphology import remove_small_objects


In [60]:
# --- Build organoid MIP ---
nuclei_mip = np.max(resampled_stack[2], axis=0)
cellmask_mip = np.max(resampled_stack[3], axis=0)

mip = nuclei_mip + cellmask_mip
viewer.add_image(mip, name="organoid_MIP")


# --- Smooth large structures ---
mip_blurred = cle.gaussian_blur(
    mip,
    sigma_x=5,
    sigma_y=5,
)
viewer.add_image(mip_blurred, name="organoid_MIP_blurred")


# --- Voronoi-Otsu: coarse organoid separation ---
organoid_seeds = cle.voronoi_otsu_labeling(
    mip_blurred,
    spot_sigma=50,     # controls seed spacing (bigger = fewer organoids)
    outline_sigma=10,  # boundary smoothness
)
viewer.add_labels(organoid_seeds, name="organoids_voronoi")


# --- Otsu mask to constrain watershed ---
organoid_mask = cle.threshold_otsu(mip_blurred)
viewer.add_labels(organoid_mask, name="organoid_mask")


# --- Distance transform inside organoid mask ---
# Ensure binary mask
organoid_mask_np = cle.pull(organoid_mask) > 0

# --- Convert data to NumPy ---
mip_blurred_np = cle.pull(mip_blurred)
organoid_seeds_np = cle.pull(organoid_seeds)

# --- Otsu mask (CPU) ---
threshold = threshold_otsu(mip_blurred_np)
organoid_mask_np = mip_blurred_np > threshold

viewer.add_labels(organoid_mask_np, name="organoid_mask")

# --- Distance transform (CPU) ---
distance_map_np = distance_transform_edt(organoid_mask_np)
viewer.add_image(distance_map_np, name="distance_map")

# --- Marker-controlled watershed (CPU) ---
# IMPORTANT: use NEGATIVE distance for watershed
organoid_labels_np = watershed(
    -distance_map_np,
    markers=organoid_seeds_np,
    mask=organoid_mask_np,
)

viewer.add_labels(organoid_labels_np, name="organoids_watershed")

# --- Cleanup ---
organoid_labels_np = remove_small_objects(
    organoid_labels_np,
    min_size=10000,
)

viewer.add_labels(organoid_labels_np, name="organoids_final")


<Labels layer 'organoids_final' at 0x1b15517f160>